In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
#Part 1

In [3]:
df_t_ = pd.read_excel("ab_testing.xlsx" , sheet_name = "Test Group") #Step1
df_k_ = pd.read_excel("ab_testing.xlsx" , sheet_name = "Control Group")

In [4]:
df_t = df_t_
df_k = df_k_

In [5]:
df_t.groupby(["Click","Purchase"]).agg({"Earning":"mean"}).head() #Step2

,,Earning
Click,Purchase,
1836.62986,822.59992,2286.71452
2268.94706,346.46262,2615.57296
2617.80386,372.12579,1947.74713
2736.41023,748.49661,2189.61267
2937.36032,531.97083,2538.08893


In [6]:
df_t["Purchase"].mean()

582.1060966484675

In [7]:
df_k.groupby(["Click","Purchase"]).agg({"Earning":"mean"}).head()

,,Earning
Click,Purchase,
2189.75316,666.48207,2111.87374
3075.31120,530.67988,2101.31948
3382.86179,315.08489,1742.80686
3555.58067,512.92875,1815.00661
3585.03168,531.73273,2185.69490


In [8]:
df_k["Purchase"].mean()

550.8940587702316

In [9]:
#Test ve kontol gruplarını incelediğimizde kontrol grubunda Click sayıları test grubuna oranla daha fazla olmasına rağmen
#ilk 5 gözlemde test grubunda Purchase ve Earning değerleri daha yüksek durmaktadır. Bunun kesinliğini AB testinden sonra 
#görebiliriz.

In [10]:
df_k["Group"]="K" #Step 3

In [11]:
df_t["Group"]="T"

In [12]:
df = pd.concat([df_k, df_t], ignore_index=True)

In [13]:
df

,Impression,Click,Purchase,Earning,Group
0,82529.45927,6090.07732,665.21125,2311.27714,K
1,98050.45193,3382.86179,315.08489,1742.80686,K
2,82696.02355,4167.96575,458.08374,1797.82745,K
3,109914.40040,4910.88224,487.09077,1696.22918,K
4,108457.76263,5987.65581,441.03405,1543.72018,K
...,...,...,...,...,...
75,79234.91193,6002.21358,382.04712,2277.86398,T
76,130702.23941,3626.32007,449.82459,2530.84133,T
77,116481.87337,4702.78247,472.45373,2597.91763,T
78,79033.83492,4495.42818,425.35910,2595.85788,T


In [14]:
#Part2

In [15]:
#Step 1
#Hipotez
#H0:M1=M2(Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
#H1:M1!=M2M1=M2(Kontrol grubu ve test grubu satın alma ortalamaları arasında fark vardır.)

In [16]:
df.groupby("Group").agg({"Purchase":"mean"})

,Purchase
Group,
K,550.89406
T,582.10610


In [17]:
#Normalization Hypothesis
test_stat, pvalue = shapiro(df.loc[df["Group"]=="K","Purchase"])
print("Test Stat = %.4f, p-value = %.4f" %(test_stat,pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [18]:
test_stat, pvalue = shapiro(df.loc[df["Group"]=="T","Purchase"])
print("Test Stat = %.4f, p-value = %.4f" %(test_stat,pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [19]:
#Normallik varsayımı testi uyguladığımızda ortaya çıkan p değerlerine göre normallik varsayımı sağlanmaktadır.
#Yani H0 reddedilemez.

In [20]:
#Variance Hypothesis
test_stat, pvalue = levene(df.loc[df["Group"]=="T","Purchase"],df.loc[df["Group"]=="K","Purchase"])
print("Test Stat = %.4f, p-value = %.4f" %(test_stat,pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [21]:
# Varyans hipotezi testi uyguladığımızda ortaya çıkan p değerlerine göre varyans hipotezi sağlanmaktadır.
#Yani H0 reddedilemez.

In [53]:
#Step 2

In [54]:
test_stat, pvalue = ttest_ind(df.loc[df["Group"]=="K","Purchase"],df.loc[df["Group"]=="T","Purchase"],equal_var=True)
print("Test Stat = %.4f, p-value = %.4f" %(test_stat,pvalue))

Test Stat = -0.9416, p-value = 0.3493


In [55]:
#Elde edilen p-value değerine göre kontrol ve test grubu ortalamaları arasında istatistiki olarak kayda değer bir fark
#yoktur. p<0.05 değerini sağlamadığı için H0 reddedilememektedir. Hipotez doğrudur.

In [56]:
#Part 3

In [57]:
#Step 1
#Her iki hipotez veriler için sağlanmaktadır. Tüm varsayımların sağlandığı durumlarda kullanacağımız test türü parametrik
# test için kullanılan "ttest_ind" olacaktır.

In [ ]:
#Step 2
#Mevcut olarak kullanılan yöntemi değiştirmek kayda değer bir satın alma farkı gözükmemektedir. Yeni yönteme ayrılacak 
#bütçenin müşteriler için kampanyalara veya reklamlara aktarılabilir.